In [2]:
!freqtrade download-data \
    --config ../config.json \
    --userdir ../ \
    --pairs BTC/USDT \
    --timeframes 1h

2026-01-06 01:20:07,697 - freqtrade - INFO - freqtrade 2026.1-dev-fc91031
2026-01-06 01:20:08,891 - freqtrade.configuration.load_config - INFO - Using config: ../config.json ...
2026-01-06 01:20:08,891 - freqtrade.loggers - INFO - Enabling colorized output.
2026-01-06 01:20:08,897 - freqtrade.loggers - INFO - Logfile configured
2026-01-06 01:20:08,897 - freqtrade.loggers - INFO - Verbosity set to 0
2026-01-06 01:20:08,897 - freqtrade.configuration.configuration - INFO - Using user-data directory: .. ...
2026-01-06 01:20:08,897 - freqtrade.configuration.configuration - INFO - Using data directory: ..\data\okx ...
2026-01-06 01:20:08,897 - freqtrade.configuration.configuration - INFO - Using pairs ['BTC/USDT']
2026-01-06 01:20:08,897 - freqtrade.configuration.configuration - INFO - timeframes --timeframes: ['1h']
2026-01-06 01:20:08,897 - freqtrade.exchange.check_exchange - INFO - Checking exchange...
2026-01-06 01:20:08,903 - freqtrade.exchange.check_exchange - INFO - Exchange "okx" is 

In [3]:
from freqtrade.data.history import load_pair_history
import pandas as pd
from pathlib import Path

df = load_pair_history(datadir=Path("../data/okx"), timeframe="1h", pair="BTC/USDT")
df.tail()

,date,open,high,low,close,volume
716,2026-01-05 12:00:00+00:00,93133.1,93215.1,92758.9,92880.7,153.590007
717,2026-01-05 13:00:00+00:00,92880.8,93015.5,92546.6,92741.8,174.540257
718,2026-01-05 14:00:00+00:00,92741.8,93681.5,92413.1,93424.7,568.059761
719,2026-01-05 15:00:00+00:00,93424.9,94010.0,93174.3,93957.1,635.309434
720,2026-01-05 16:00:00+00:00,93957.0,94077.7,93587.0,93709.1,332.503568


In [4]:
from freqtrade.plot.plotting import generate_candlestick_graph
pair = "BTC/USDT"#绘图
graph = generate_candlestick_graph(data=df, pair=pair)
graph.show()



No trades found.


In [8]:
from freqtrade.plot.plotting import generate_candlestick_graph
import talib
import numpy as np

pair = "BTC/USDT"

close = df["close"].to_numpy(dtype=np.float64)

df["ema20"] = talib.EMA(close, timeperiod=20)
df["ema144"] = talib.EMA(close, timeperiod=144)
df["ema169"] = talib.EMA(close, timeperiod=169)
df["ema576"] = talib.EMA(close, timeperiod=576)
df["ema676"] = talib.EMA(close, timeperiod=676)
df["rsi"] = talib.RSI(close, timeperiod=14)

# 绘图
graph = generate_candlestick_graph(
    data=df,
    pair=pair,
    indicators1=["ema20", "ema144", "ema169", "ema576", "ema676"],
    indicators2=["rsi"],
)
graph.show()

No trades found.
